In [4]:
import pandas as pd

# Load raw data
df = pd.read_csv('/home/kamiar/chevron/Acid-neural-net/data/test_data_nn(SampleData_07032025_vf3).csv')

# Define columns by position
input_cols = df.columns[3:20]             # Inputs 1–17
f4_col = df.columns[24]                   # F4 → input_18
output_col = df.columns[20]               # Output
quality_cols = df.columns[22:24]          # F2, F3

# Subset and rename
df_selected = df[list(input_cols) + [f4_col] + [output_col] + list(quality_cols)]
df_selected = df_selected.rename(columns={f4_col: 'input_18'})

# Identify and remove rows with >1 missing value
missing_counts = df_selected.isna().sum(axis=1)
removed_indices = missing_counts[missing_counts > 1].index.tolist()
print("Removed row indices:", removed_indices)

# Keep only rows with at most one missing
df_cleaned = df_selected[missing_counts <= 1]

# Preview cleaned data
print("\nCleaned data preview (first 5 rows):")
print(df_cleaned.head())

# Save cleaned dataset
output_path = '/home/kamiar/chevron/Acid-neural-net/data/test_data_nn(SampleData_07032025_vf3)_cleaned.csv'
df_cleaned.to_csv(output_path, index=False)
print(f"\nCleaned CSV saved to: {output_path}")


Removed row indices: [70, 131, 135, 145, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161]

Cleaned data preview (first 5 rows):
   Input 1  Input 2  Input 3  Input 4  Input 5  Input 6  Input 7  Input 8  \
0      2.0      1.0      3.0      1.0      5.0      1.0      2.0      9.0   
1      2.0      1.0      3.0      1.0      5.0      2.0      1.0      NaN   
2      2.0      2.0      5.0     16.0      4.0      1.0      1.0      5.0   
3      2.0      2.0      5.0      1.0      5.0      1.0      2.0      5.0   
4      1.0      5.0      4.0     15.0      5.0      1.0      2.0      9.0   

  Input 9  Input 10  ...  Input 12  Input 13  Input 14  Input 15  Input 16  \
0      15       7.0  ...       8.9     337.0  0.000900   47360.0     306.3   
1      13      48.0  ...     201.6      17.0  0.000909    2319.0      15.1   
2      20     451.0  ...     150.3    1514.0  0.000918  211636.0    1379.3   
3      15     100.0  ...      19.0    1662.0  0.000927  231753.0    1534.2   
4  